# IPython Notebookの拡張機能live_revealについて

## live_revealについて

IPython Notebookは、そのままでも十分に活用の方法がありますが、拡張機能を用いることによってさらに多くのことを実行することができるようになります。例えば、[Damián Avila](https://github.com/damianavila)さんが開発している[live_reveal](https://github.com/damianavila/live_reveal)は、JavaScriptによってHTMLプレゼンテーションを行える[Reveal.js](http://lab.hakim.se/reveal-js/#/)を利用し、Notebook環境用のスライドショーの作成と表示を可能にします。この拡張機能をインストールすると、Notebookのツールボタンの一番右横にスライドボタンが追加され、このボタンを押すことによって、HTMLにレンダリングされた内容がReveal.jsの力を借りてスライドショーとして表示することができます。また、表示しながらリアルタイムにコードを実行することもでき、下の動画などではその様子を実際に見ることができます。

<img src=http://nbviewer.ipython.org/github/fperez/nb-slideshow-template/blob/master/toolbar-slideshow.png />

Damián AvilaさんのScipy 2014での講演の様子は以下の動画から見ることができます(画像をクリックするとYoutubeのページに飛びます)。

[![RJSE/RISE video](http://img.youtube.com/vi/sZBKruEh0jI/0.jpg)](https://www.youtube.com/watch?v=sZBKruEh0jI)

## インストール方法

拡張機能のインストールの仕方は、GitHubのREADMEにも書いてありますが、

    git clone https://github.com/damianavila/live_reveal.git
    
で自分のPCにリポジトリをクローンしてきて、その中で

    python setup.py install

とすれば、`~/.ipython/nbextension`の中に`livereveal`というディレクトリが新たに作成されているはずです。

`nbextension`が存在しないときには、IPythonのバージョンが2以降になっているか確認してください。IPythonのアップデートは、

    sudo pip install -U "ipython[all]"

とすればよいです。

## 使い方(初級編)

### セルの設定

実際にスライドを作る際には、ツールバーの"Cell Toolbar"のスピンボタンから"Slideshow"を選択した状態で行える、各セルのメタデータの編集を行う必要があります。各セルの右上に"Slide Type"というのが表示されますので、

- Slide: 横方向のスライド
- Sub-Slide: 縦方向のスライド
- Fragment: 同じスライド内で後から表示するもの
- Skip: スライドショー時に無視される
- Note: スライドショー時に無視され、ノートとして表示する

というタイプに割り当てることができます。大きな章ごとにスライドをつくり、その下に内容を入れていく、といった使い方もできますし、基本的にはすべて"Slide"でつくり、補足資料を"Sub-Slide"にする、という使い方もあると思います。

### テーマの選択

また、設定ファイルを変更することによって、スライドショー時の見た目を変更することができます。まず、簡単に変更できるものから紹介します。

プロファイルの中のjsファイルを書き換えることによって、はじめから用意されているテーマや遷移アニメーションを適用することができます。

`~/.ipython/profile_プロファイル名/static/custom/custom.js`の中を編集します。最後の方に

```js
$([IPython.events]).on('app_initialized.NotebookApp', function(){

     require(['nbextensions/livereveal/main'],function(livereveal){
       // livereveal.parameters('theme', 'transition', 'fontsize', static_prefix);
       //   * theme can be: simple, sky, beige, serif, solarized
       //   (you will need aditional css for default, night, moon themes).
       //   * transition can be: linear, zoom, fade, none
       livereveal.parameters('simple', 'zoom');
       console.log('Live reveal extension loaded correctly');
     });

});
```

のようにあるので、指示通り`livereveal.parameters('simple', 'zoom')`のパラメータの値を変えることで、Notebookをリロードすれば選択したテーマやアニメーションを使うことができます。普通のパワーポイントと同じように、あまり過剰な色付けやアニメーションは嫌われる傾向があるので(特にアカデミック分野では)、"`simple`"の"`fade`"あたりがFAではないでしょうか。

### 細かいTips

####数式は細かく分割する

数式はほとんど画像と同じように扱われてしまうので、あまり長いと、スライドの端を超えてしまい、内容が読めなくなってしまう場合があります。したがって、できるだけ数式は横に長くならないようにすること。コンマでつなぐ部分はコンマは外に出して一個ずつ独立な数式にすること、こうすると適当な位置で折り返してくれるようになります。$\LaTeX$の`align`クラスをうまく使って縦に伸ばすこと。

####図や文章を中心に揃える

中心に揃えたい部分を
>`<div align="center"> ・・・ </div>`

と囲んでやれば中心に表示されるようになります。とは言いましたが、実際には右にすこしずれてしまいます。この問題の解決法は次の中級編で。

## 使い方(中級編)

次に、画面サイズとの関係など、実際的ですが、なかなか調整の難しい部分のカスタマイズについて述べていくことにします。現在使いながらの更新なので、後から変更する点も多いと思われますが、暫定の使い方ということでまとめておきます。FAQのような形にしていきます。

### `<div align="center">`使っとるのに右にズレとるやんけ!

これにだいぶ悩まされました。この原因は、それぞれのセルの実行番号を表示する領域が左に取られているためです。したがって、この領域を消してやる必要があります。ちょっとググると

- [IPython Slideshows will change the way you work | Hannes Bretschneider:](http://hannes-brt.github.io/blog/2013/08/11/ipython-slideshows-will-change-the-way-you-work/)

のような情報が見つかります。上の例では、`nbconvert`でスライドにする際にインプットセルと、プロンプト領域を消す方法が載っています。このスクリプトを試しにそのまま(`<script>`タグは当然外して)`~/.ipython/nbextensions/main.js`の中に書き込んで再読み込みすると、確かにインプットセルとプロンプト領域が表示されなくなります。しかし、これだと編集の際や、他の普通の使い方をしたいときに困るので、スライドショーの表示をするときだけ有効にしたいと考えます。

IPythonではプロファイルといって、ある設定群を指定して起動することができるので、その1つに、今回のようなスライド用の設定を作ってしまえばいいことが分かります。

    ipython profile create slide

などとすれば、`~/.ipython`以下に"profile_slide"という新しいプロファイルができていることが確認できるかと思います。

そこで`~/.ipython/profile_slide/static/custom/custom.js`の中身を以下のように編集します。

```js
$([IPython.events]).on('app_initialized.NotebookApp', function(){

     require(['nbextensions/livereveal/main'],function(livereveal){
       // livereveal.parameters('theme', 'transition', 'fontsize', static_prefix);
       //   * theme can be: simple, sky, beige, serif, solarized
       //   (you will need aditional css for default, night, moon themes).
       //   * transition can be: linear, zoom, fade, none
       livereveal.parameters('simple', 'fade');
       console.log('Live reveal extension loaded correctly');
    // http://hannes-brt.github.io/blog/2013/08/11/ipython-slideshows-will-change-the-way-you-work/
    function hideElements(elements, start) {
        for(var i = 0, length = elements.length; i < length;i++) {
            if(i >= start) {
                elements[i].style.display = "none";
            }
        }
    }
    var input_elements = document.getElementsByClassName('input');
    hideElements(input_elements, 0);
    var prompt_elements = document.getElementsByClassName('prompt');
    hideElements(prompt_elements, 0);
     });

});
```

このように編集してページをリロードすると、先程試してみたときのように、インプットセルとプロンプト領域が表示されなくなっていることが分かるかと思います。

### 字を大きくしたいんですが...

これは多分みなさん感じるでしょう。自分のブラウザ上で表示する分には綺麗に表示できているのに、いざプロジェクターに映してみると、いろんな人から「字が小さくて見えない」と指摘を受けます。ブラウザの機能で拡大表示すると、今度は図も大きくなってしまったりして、うまく行きません。

簡単に解決できます。

`~/.ipython/nbextensions/livereveal/main.css`の14行目辺りを

```css
.reveal .slides {
  text-align: left;
  font-size: 190%; //160%
  //width: 100% !important; //960px, 1366px
}
```

のように編集します。widthの項は無視してください。font-sizeが160%だったものを、190%に変更しています。このようにすることで、大体ブラウザ上で125%に拡大した時と同じくらいの大きさの文字サイズに変更することができ、しかも文字以外のオブジェクトのサイズはそのままにできます。

### 左の二つのボタンを消してやりたい

強引に消すやり方は、これらを定義している箇所、`~/.ipython/nbextensions/livereveal/main.js`の305行目辺りの`buttonExit`の定義のなかで`$('.reveal').after(help_button);`となっている行を"`//`"でコメントアウトすればと良いです。同じようにその後に定義されている`buttonExit`の中の同様の部分をコメントアウトすれば、毎回これらのボタンを表示しないようにすることができます。

また、プロファイルによって表示させたり表示させなかったりしたい場合には、はじめの例のように、プロファイルの`custom.js`の中に

```js
    var buttonHelp_elements = document.getElementsByClassName('buttonHelp');
    hideElements(buttonHelp_elements, 0);
    var buttonExit_elements = document.getElementsByClassName('buttonExit');
    hideElements(buttonExit_elements, 0);
```

のように、表示しないものに追加してしまえば良いです。このようにすれば、特定のプロファイル(本番用)ではこれらのボタンを表示しないようにすることができます。

## 今のところどうした良いかわからない問題

ここから先は、現状どうしたらよいかまだわかっていない問題に関して書き留めておき、議論がされていればそこのリンクを見るなどして解決に向けて知識を集めていくための場所にします。

既知の問題として、スライドをpdfにするのはうまく行きません。Firefoxで印刷でpdfエクスポートにすると、下の領域を含んだ形でpdf化しようとしてくれます。Chromeではどうかというと、そこまでひどくはないですが、図のインポートを行うと、拡大して表示されて欲しい位置に、小さい図がいくつも詰められているような残念なpdfがエクスポートされます。したがって、この問題を解決する1つの方法は、nbconvertで一旦slideにしてからそれをブラウザ上でpdfにする方法です。しかし、筆者の藤本はこのnbconvertの設定で割と躓いてしまったので、結局すべてスクリーンショットに撮って、それをconvertで1つのpdfにまとめるという暴挙に出ました。このときに注意するポイントとしては、実際にプレゼンで使うであろうプロジェクターの画面サイズに自分のPCを合わせておいてからスクリーンショットを撮ることです。こうしないと、ワイドなスライドができたり、余白の大きすぎるスライドが量産されることとなります。後学のために、実際に使ったコマンド、ツール類を記録しておきます。

スクリーンショットはshutterというアプリケーションで、選択範囲のスクリーンショットを撮りました。これを1つのディレクトリ内にスライドの若いものがファイル名の数字の小さいものになるように注意しながら1枚ずつ保存していきます。すべてのスライドが保存し終わったら、そのディレクトリ内で

    convert -page 1024x768 -gravity Center *.jpg out.pdf
    
と実行します。これによって横1024px, 縦768px(←研究室のプロジェクターの解像度)で、すべてのjpg画像ファイルが数字の順番に並べられ、中心に揃えられたout.pdfが作成されます。すべて画像として扱っているのでファイルサイズが大きくなるのはご愛嬌。文字が微妙に潰れて見えにくのもご愛嬌。印刷して小さくなれば目立たないはずなんや! ただし、予備として準備するスライドはこれではよくない気がする。予備のpdfファイルは、おとなしくPowerPoint使って一から作った方がいいのかもしれないですね。